In [ ]:
%pip install -r requirements.txt

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm


from backtesterClass.orderClass import orders
from backtesterClass.orderBookClass import OBData
from backtesterClass.analysisClass import analysisClass


from strats.LTSMOnlineLearn import LTSMOnlineLearnStrat


from utils.utils import global_perf, peformance_metrics

from cProfile import Profile
from pstats import SortKey, Stats
import gc


# Improve computationnal performance of the backtester - increases garbage collector threshold

allocs, gen1, gen2 = gc.get_threshold()
allocs = 5000
gen1 = gen1*2
gen2=gen2*2
gc.set_threshold(allocs, gen1,gen2)

## Load and Initialize data

In [9]:
db_path = r"data/all_ohlcv_data.csv"
dataClass = OBData(db_path)

## LTSM Strategy

•⁠  LTSM strategy:
  - LTSM: short_window=5, long_window=20, RSI_window=14, buyThreshold=30, sellThreshold=70, lag_day1, lag_day2, lag_day5, lag_day10, alpha=0.05 ,trainLen = 200, forecast=2

In [10]:
LTMSOnlineTrader1 = LTSMOnlineLearnStrat("LTMSOnlineTrader1", short_window=5, long_window=20, RSI_window=14, buyThreshold=30, sellThreshold=70, alpha=1/14 ,trainLen = 200, forecast=2)
LTMSOnlineTrader2 = LTSMOnlineLearnStrat("LTMSOnlineTrader2", short_window=5, long_window=20, RSI_window=50, buyThreshold=25, sellThreshold=75, alpha=1/50 ,trainLen = 200, forecast=2)
LTMSOnlineTrader3 = LTSMOnlineLearnStrat("LTMSOnlineTrader3", short_window=50, long_window=200, RSI_window=14, buyThreshold=30, sellThreshold=70, alpha=1/14 ,trainLen = 200, forecast=2)
LTMSOnlineTrader4 = LTSMOnlineLearnStrat("LTMSOnlineTrader4", short_window=50, long_window=200, RSI_window=50, buyThreshold=25, sellThreshold=75, alpha=1/50 ,trainLen = 200, forecast=2)


['cumulative returns', 'lag_ret_1', 'lag_ret_10', 'lag_ret_2', 'lag_ret_5', 'long_ma', 'rsi', 'short_ma'] | StandardScaler | RollingRegressor
['cumulative returns', 'lag_ret_1', 'lag_ret_10', 'lag_ret_2', 'lag_ret_5', 'long_ma', 'rsi', 'short_ma'] | StandardScaler | RollingRegressor
['cumulative returns', 'lag_ret_1', 'lag_ret_10', 'lag_ret_2', 'lag_ret_5', 'long_ma', 'rsi', 'short_ma'] | StandardScaler | RollingRegressor
['cumulative returns', 'lag_ret_1', 'lag_ret_10', 'lag_ret_2', 'lag_ret_5', 'long_ma', 'rsi', 'short_ma'] | StandardScaler | RollingRegressor


## Backtesting 

In [11]:
for _ in tqdm(range(len(dataClass.OBData_))):
    orderClass = orders()
    LTMSOnlineTrader1.strategy(orderClass)
    LTMSOnlineTrader2.strategy(orderClass)
    LTMSOnlineTrader3.strategy(orderClass)
    LTMSOnlineTrader4.strategy(orderClass)
    OBData.step +=1 

  0%|          | 10/5031 [00:00<00:51, 96.97it/s]/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/deep_river/regression/regressor.py:152: UserWarning:

Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

100%|██████████| 5031/5031 [4:27:45<00:00,  3.19s/it]  


## Single Stock and Global Analysis - LTSM

In [12]:
analysisLTSM = analysisClass(LTMSOnlineTrader1, path=r"Results", dashboardName="executionLTSMStrat", dbName="dbLTSMStrat")
analysisLTSM.create_dashboard("AAPL", show=True)

In [14]:
# Strategy instances
strategies = {
    "LTMSOnlineTrader1": LTMSOnlineTrader1,
    "LTMSOnlineTrader2": LTMSOnlineTrader2,
    "LTMSOnlineTrader3": LTMSOnlineTrader3,
    "LTMSOnlineTrader4": LTMSOnlineTrader4,

}

global_perf(strategies, "LTSM")

peformance_metrics(strategies, verbose=True)


,Strategy,Sharpe Ratio (annualized),Max Drawdown (%),Total PnL ($m),Total Return (%),Annualized Return (%),Volatility (%),Avg PnL/Asset ($),Avg Variance/Asset,Turnover
0,LTMSOnlineTrader1,0.001113,-37.407968,-5.454895,-5.307895,-0.272813,7.502364,-25137.766895,3.716279e+11,104.297041
1,LTMSOnlineTrader2,-0.107970,-45.417858,-22.009234,-22.204234,-1.249786,8.384231,-101425.041605,5.174676e+11,72.539304
2,LTMSOnlineTrader3,0.119938,-26.516000,13.005662,12.934662,0.611144,7.311788,59933.927432,2.039334e+11,74.932903
3,LTMSOnlineTrader4,0.048268,-34.519978,1.405723,1.122723,0.055939,8.303972,6477.986001,3.299157e+11,64.370959
